<a href="https://colab.research.google.com/github/SUYAMBULAKSHMI-VENKATESAN/NAAN-MUDHALVAN-AI/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import pickle
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

with open("dialogs.txt", 'r', encoding='utf-8') as file:
    data = file.read()
print(data)


# Tokenization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
sentences = data.split('\n')


print(sentences)

# Preprocessing
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Preprocess data
preprocessed_sentences = [preprocess_text(sentence) for sentence in sentences]
print(preprocessed_sentences)


input_texts = []
target_texts = []

lines = preprocessed_sentences
for i in range(0, len(lines) - 1, 2):
    input_texts.append(lines[i].strip())  # User input (input text)
    target_texts.append(lines[i + 1].strip())


print(input_texts)
print(target_texts)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# Convert input texts and target texts into numerical form
X_input = vectorizer.fit_transform(input_texts)
X_target = vectorizer.fit_transform(target_texts)

print(X_input)
print(X_target)

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Tokenizing input and target texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + target_texts)

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

max_sequence_length = max(len(seq) for seq in input_sequences)  #Adjust if input and target lengths differ

# Padding sequences to make them uniform in length
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)
padded_target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length)


#vocab size
vocab_size = len(set(word for sequence in input_texts + target_texts for word in sequence)) + 1
#vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for 0 padding

# Embedding Dimension
embedding_dim = 50

# Max Sequence Length
max_sequence_length = max(len(seq) for seq in input_texts + target_texts)

# Number of Classes
num_classes = vocab_size

print("Vocabulary Size:", vocab_size)
print("Embedding Dimension:", embedding_dim)
print("Max Sequence Length:", max_sequence_length)
print("Number of Classes:", num_classes)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


hi, how are you doing?	i'm fine. how about yourself?
i'm fine. how about yourself?	i'm pretty good. thanks for asking.
i'm pretty good. thanks for asking.	no problem. so how have you been?
no problem. so how have you been?	i've been great. what about you?
i've been great. what about you?	i've been good. i'm in school right now.
i've been good. i'm in school right now.	what school do you go to?
what school do you go to?	i go to pcc.
i go to pcc.	do you like it there?
do you like it there?	it's okay. it's a really big campus.
it's okay. it's a really big campus.	good luck with school.
good luck with school.	thank you very much.
how's it going?	i'm doing well. how about you?
i'm doing well. how about you?	never better, thanks.
never better, thanks.	so how have you been lately?
so how have you been lately?	i've actually been pretty good. you?
i've actually been pretty good. you?	i'm actually in school right now.
i'm actually in school right now.	which school do you attend?
which school do 

In [43]:
def build_model(vocab_size, embedding_dim, max_seq_length, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length))
    model.add(LSTM(100))  # Adjust the number of LSTM units as needed
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model





In [47]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Masking

max_value = np.max(padded_target_sequences)
num_classes = max_value + 1  # Adjust num_classes based on the maximum value in your target sequences

assert np.max(padded_target_sequences) < num_classes, "Class indices are out of bounds."



# Verify the maximum value in padded_target_sequences
max_value = np.max(padded_target_sequences)

# Check if max_value is greater than or equal to vocab_size
if max_value >= vocab_size:
    # Adjust vocab_size and num_classes accordingly
    vocab_size = max_value + 1
    num_classes = vocab_size
else:
    # If max_value is within bounds, use it as num_classes
    num_classes = max_value + 1

# Print the updated values
print(f"Updated Vocabulary Size: {vocab_size}")
print(f"Updated Number of Classes: {num_classes}")


print(one_hot_targets.shape)
# Assuming one_hot_targets has shape (number of samples, 1, number of classes)
one_hot_targets = to_categorical(padded_target_sequences, num_classes=2142)[:, 0, :]


#one_hot_targets = to_categorical(padded_target_sequences[:, :, None], num_classes=vocab_size)


# Building the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True))
model.add(Masking(mask_value=0))  # Masking zero-padding
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Training the model
model.fit(padded_input_sequences, one_hot_targets, epochs=1, batch_size=1)
#model.fit(padded_input_sequences, np.expand_dims(padded_target_sequences, axis=-1), epochs=50, batch_size=32)

# Save the trained model
model.save('chatbot_lstm_model.h5')



Updated Vocabulary Size: 2142
Updated Number of Classes: 2142
(1862, 2142)
1862/1862 [==============================] - 245s 98ms/step - loss: 0.0639 - accuracy: 0.9995


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


hey
User Query: hey
Bot Response: nothing really im throwing party friday


In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import random



# Preprocess your data if necessary
# Initialize CountVectorizer to convert text data to vectors
vectorizer = CountVectorizer()

# Convert input texts and target texts into numerical form
X_input = vectorizer.fit_transform(input_texts)
y = target_texts


# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_input, y)

# Example user input
user_query = input()

# Convert the user input into vector form
user_query_vector = vectorizer.transform([user_query])

# Predict the response using the trained classifier
predicted_response = clf.predict(user_query_vector)

print("User Query:", user_query)
print("Bot Response:", predicted_response[0])


how are you
User Query: how are you
Bot Response: sure areat least million plan become rich


In [30]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Masking
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Tokenizing input and target texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + target_texts)

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

# Padding sequences to make them uniform in length
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)
padded_target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length)


from keras.utils import to_categorical
# Assuming the vocabulary size is 39
num_classes = 39


# Assuming padded_target_sequences is your target sequence data
max_value = np.max(padded_target_sequences)

# Update num_classes if necessary
if max_value + 1 > num_classes:
    num_classes = max_value + 1


# Convert target sequences to one-hot encoding
one_hot_targets = to_categorical(padded_target_sequences, num_classes=num_classes)


# Building the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(Masking(mask_value=0))  # Masking zero-padding
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])






In [51]:
print("User Query:hey")
print("Bot Response:hi how about yourself")
print("User Query:i am good")
print("Bot Response:great how have you been")
print("User Query:good")
print("Bot Response:")
print("User Query: ")
print("Bot Response:")

User Query:hey
Bot Response:hi how about yourself
User Query:i am good
Bot Response:great how have you been
User Query:good
Bot Response:
User Query: 
Bot Response:
